In [1]:
# import dataset using dataloader_cifar
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import Dataloaders.dataloader_cifar10
import sys
import torchsummary
# use the cifar 10 to load the data
train_loader, test_loader, val_loader, classes = Dataloaders.dataloader_cifar10.Dataloader_cifar10(32, 2024)
# define the network, we use the mobilenet_v2
from Models.mobilenetv2 import MobileNetV2
model = MobileNetV2()
client_model = model.get_client_model()
server_model = model.get_server_model()
model = nn.Sequential(client_model, server_model)
model = model.cuda()
# print the model summary
print('client_model')
print(client_model)
print('server_model')
print(server_model)
print('model')
print(model)
base_params = [params for name, params in model.named_parameters() if ("xxx" in name)]
finetune_params = [params for name, params in model.named_parameters() if ("yyy" in name)]
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

for epoch in range(20):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        images, gts = batch[0].cuda(), batch[1].cuda()
        # print the shape of wegihts and bias

        preds = model(images)
        loss = F.cross_entropy(preds, gts)
        optimizer.zero_grad()    
        loss.backward()          
        optimizer.step()         
        epoch_loss += loss * train_loader.batch_size
        # 计算其他标准
    loss = epoch_loss / len(train_loader.dataset)
    print(f"Epoch {epoch} loss: {loss}")
    # .......
    # 每隔几个epoch在测试集上跑一下
    if epoch % 5 == 0:
        model.eval()
        test_epoch_loss = 0
        for test_batch in test_loader:
            test_images, test_gts = test_batch[0].cuda(), test_batch[1].cuda()
            test_preds = model(test_images)
            loss = F.cross_entropy(test_preds, test_gts)
            test_epoch_loss += loss * test_loader.batch_size
            # 计算其他标准
        test_loss = test_epoch_loss / (len(test_loader.dataset))
        print(f"Epoch {epoch} train loss: {loss}, test loss: {test_loss}")
    # .......
    # 根据条件对指定epoch的模型进行保存 将模型序列化到磁盘的pickle包中



Files already downloaded and verified
Files already downloaded and verified


/home/tonypeng/anaconda3/envs/iot/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [10]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', weights=True)


Using cache found in /home/tonypeng/.cache/torch/hub/pytorch_vision_v0.10.0


In [24]:
model_slices = []

slices = [x for x in model.modules() if not isinstance(x, nn.Sequential)]

In [50]:
ll = model.features
ll
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [56]:
from Models.mobilenetv2 import MobileNetV2
MobileNetV2 = MobileNetV2()


/home/tonypeng/anaconda3/envs/iot/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tonypeng/anaconda3/envs/iot/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


159